In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


df = pd.read_csv("emotions.csv")
df

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...
416804,i feel like telling these horny devils to find...,2
416805,i began to realize that when i was feeling agi...,3
416806,i feel very curious be why previous early dawn...,5
416807,i feel that becuase of the tyranical nature of...,3


In [2]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

In [3]:
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [4]:
svm = LinearSVC()
svm.fit(X_train_tfidf, y_train)

LinearSVC()

In [5]:
y_pred = svm.predict(X_test_tfidf)

label_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

report = classification_report(y_test, y_pred, target_names=label_names)
print(report)

              precision    recall  f1-score   support

     sadness       0.94      0.93      0.93     24201
         joy       0.91      0.92      0.91     28164
        love       0.78      0.77      0.78      6929
       anger       0.90      0.90      0.90     11441
        fear       0.84      0.84      0.84      9594
    surprise       0.72      0.70      0.71      3033

    accuracy                           0.89     83362
   macro avg       0.85      0.84      0.85     83362
weighted avg       0.89      0.89      0.89     83362



In [6]:
test_sentence = "I can't stop smiling today, everything feels amazing!"
test_vector = vectorizer.transform([test_sentence])

predicted_label = svm.predict(test_vector)[0]

print(f"Predicted emotion: {label_names[predicted_label]}")

Predicted emotion: joy
